## Mono-objective Algorithms versus Optimal Solutions

Tool to benchmark mono-objective algorithms against the optimal solutions of three reduced datasets of 15 students, maintaining the demographic distribution of the original dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import contextlib
import io
import datetime
import import_ipynb

import standard_genetic_algorithm as sga
import enhanced_genetic_algorithm as ega
import particle_swarm_optimization as pso
import utils.fitness_functions as ff
import utils.monoobjective_exhaustive_solver as es

import os
os.makedirs("figures", exist_ok=True)

RUNS = 100
REDUCED_DATASETS = {
    "Dataset 1": "data/reduced_dataset_1.csv",
    "Dataset 2": "data/reduced_dataset_2.csv",
    "Dataset 3": "data/reduced_dataset_3.csv"
}

for label, path in REDUCED_DATASETS.items():
    dataset_df = pd.read_csv(path)
    sga.DATASET = dataset_df
    ega.DATASET = dataset_df
    pso.DATASET = dataset_df

    optimal_arrangement, optimal_score, _ = es.find_best_arrangement(dataset_df, path)

    sga_scores_runs, ega_scores_runs, pso_scores_runs = [], [], []

    with contextlib.redirect_stdout(io.StringIO()):
        for _ in tqdm(range(RUNS), desc=label):

            timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")

            sga.RUN_TIME = timestamp
            sga.BASE_DIR = f"output/mono_vs_optimal/sga/{label.lower().replace(' ', '_')}"
            df_sga = sga.execute()
            sga_scores_runs.append(df_sga["score"])
            
            ega.RUN_TIME = timestamp
            ega.BASE_DIR = f"output/mono_vs_optimal/ega/{label.lower().replace(' ', '_')}"
            df_ega, _ = ega.execute()
            ega_scores_runs.append(df_ega["score"])

            pso.RUN_TIME = timestamp
            pso.BASE_DIR = f"output/mono_vs_optimal/pso/{label.lower().replace(' ', '_')}"
            df_pso, _ = pso.execute()
            pso_scores_runs.append(df_pso["score"])

    # Convert to plain NumPy arrays to prevent shape issues
    sga_avg = pd.concat(sga_scores_runs, axis=1).mean(axis=1).to_numpy()
    ega_avg = pd.concat(ega_scores_runs, axis=1).mean(axis=1).to_numpy()
    pso_avg = pd.concat(pso_scores_runs, axis=1).mean(axis=1).to_numpy()
    gens_iters = np.arange(len(sga_avg))

    # Plot each dataset separately
    plt.figure(figsize=(10, 6))
    plt.plot(gens_iters, sga_avg, label="SGA", linestyle="--", color="black")
    plt.plot(gens_iters, ega_avg, label="EGA", linestyle="-.", color="dimgray")
    plt.plot(gens_iters, pso_avg, label="PSO", linestyle="-", color="gray")
    plt.hlines(optimal_score, gens_iters[0], gens_iters[-1], colors="red", linestyles=":", label="Optimal")

    plt.title(f"Reduced {label} — SGA, EGA & PSO vs. Optimum (Mean of {RUNS} Run{'s' if RUNS > 1 else ''})")
    plt.xlabel("Generation / Iteration")
    plt.ylabel("Fitness Score")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"figures/fig_{label.lower().replace(' ', '_')}_ga_pso_vs_optimal.png")
    plt.show()